In [1]:
import torch
import os
import pandas as pd
from torch.utils.data import Dataset
from skimage import io
import torchvision.transforms as transforms
import cv2
import nltk
from nltk import tokenize
from torch.utils.data import DataLoader

import torchvision.models as models
import torch.nn as nn

from sklearn.model_selection import train_test_split

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
print(device)

cuda


In [4]:
# class mydataset(Dataset):
#     def __init__(self,csv_file,root_dir,transform=None):
#         self.annotations = pd.read_csv(csv_file)
#         self.root_dir = root_dir
#         self.transform = transform
        
#     def __len__(self):
#         return len(self.annotations)
    
#     def __getitem__(self,index):
#         img_path = os.path.join(self.root_dir,self.annotations.iloc[index,0])
#         image = io.imread(img_path)
#         y_label = tokenize.word_tokenize(self.annotations.iloc[index,1])
#         y_label = torch.tensor([int(i) for i in y_label])
#         one_hot = torch.zeros(19)
#         one_hot[y_label.add(-1)] = 1.0
#         y_label = one_hot
#         #y_label = self.annotations.iloc[index,1].split(' ')
#         #y_label = torch.tensor(y_label)
#         #y_label = torch.tensor(int(self.annotations.iloc[index,1]))
#         #y_label = y_label.Split(' ').Select(Int32.Parse).ToList()
        
#         if self.transform:
#             image = self.transform(image)
            
#         return(image,y_label)

In [4]:
transforming = transforms.Compose([
    transforms.ToPILImage(),
    #transforms.Scale(256),
    #transforms.Grayscale(num_output_channels = 1),
    transforms.Resize((256,256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                        std = [0.229,0.224,0.225])
])

transforming2 = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256,256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                        std = [0.229,0.224,0.225])
])

# datasets = mydataset(csv_file = 'train.csv',
#                     root_dir = 'C:/Users/Bob/Desktop/Deep Learning/Data/COMP5329S1A2Dataset/data', transform = transforming)

In [5]:
#################################################################################################################
class mydataset2(Dataset):
    def __init__(self,df,root_dir,transform=None):
        self.annotations = df
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):  #return the length of our data
        return len(self.annotations)
    
    def __getitem__(self,index): #this will tell our model how to get a single sample of our data, index is some number less than length
        # index is get a placeholder for a single sample
        img_path = os.path.join(self.root_dir,self.annotations.iloc[index,0])
        image = io.imread(img_path)
        y_label = tokenize.word_tokenize(self.annotations.iloc[index,1])
        y_label = torch.tensor([int(i) for i in y_label])
        one_hot = torch.zeros(19)
        one_hot[y_label.add(-1)] = 1.0
        y_label = one_hot
        #y_label = self.annotations.iloc[index,1].split(' ')
        #y_label = torch.tensor(y_label)
        #y_label = torch.tensor(int(self.annotations.iloc[index,1]))
        #y_label = y_label.Split(' ').Select(Int32.Parse).ToList()
        
        if self.transform:
            image = self.transform(image)
            
        return(image,y_label)

In [6]:
df = pd.read_csv("train.csv")
df
#trainsss,testsss = torch.utils.data.random_split(df,[25000,5000])
trainsss,testsss = train_test_split(df,test_size=0.15)
traindatasets = mydataset2(trainsss,
                    root_dir = 'C:/Users/Bob/Desktop/Deep Learning/Data/COMP5329S1A2Dataset/data', transform = transforming)
validdatasets = mydataset2(testsss,
                    root_dir = 'C:/Users/Bob/Desktop/Deep Learning/Data/COMP5329S1A2Dataset/data', transform = transforming2)

# for x,y in traindatasets:
#     print(x.shape)
#     break
for a,b in traindatasets:
    print(a.shape)
    print(b)
    break
print(len(traindatasets))
print(len(validdatasets))

torch.Size([3, 224, 224])
tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.])
25500
4500


In [7]:
df = pd.read_csv("train.csv")
df

,ImageID,Labels,Caption
0,0.jpg,1,Woman in swim suit holding parasol on sunny day.
1,1.jpg,1 19,A couple of men riding horses on top of a gree...
2,2.jpg,1,They are brave for riding in the jungle on tho...
3,3.jpg,8 3 13,a black and silver clock tower at an intersect...
4,4.jpg,8 3 7,A train coming to a stop on the tracks out side.
...,...,...,...
29995,29995.jpg,8 1 2,A picture of a truck that is in the middle of ...
29996,29996.jpg,1,A plate topped with a pizza being cut with a s...
29997,29997.jpg,1,A man riding a snowboard on top of snow.
29998,29998.jpg,1,This photo shows people skiing in the mountains.


In [6]:
############################################################################################################
# for x,y in datasets:
#     print(x.shape)
#     print(y)
#     break

torch.Size([3, 224, 224])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.])


In [7]:
#len(datasets)

30000

In [8]:
#train,test = torch.utils.data.random_split(datasets,[25000,5000])

In [8]:
batch_size=25
training = DataLoader(traindatasets,batch_size=batch_size,shuffle=True,num_workers=0,pin_memory=True)
validation = DataLoader(validdatasets,batch_size=batch_size)#,num_workers=1,pin_memory=True)

In [9]:
[x for x in dir(models)]
print("xxx")

xxx


In [10]:
mymodel = models.densenet201(pretrained=True)

In [11]:
for x in mymodel.parameters():
    x.require_grad=False

In [13]:
mymodel

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [12]:
class dense(nn.Module):
    def __init__(self):
        super(dense,self).__init__()
        self.network = nn.Sequential(mymodel.train(),
                                    nn.Linear(1000,500,bias=True),
                                    nn.Linear(500,19))
    def forward(self,data):
        out = self.network(data)
        
        return out
        

In [21]:
####saving and laoding mymodel parameters####################
def save_checkpoint(state,filename="assignment2_rework.pth.tar"):
    print(":) Saving checkpoint")
    torch.save(state,filename)

def load_checkpoint(mycheckpoint):
    print(":) loading checkpoint")
    testmodel.load_state_dict(mycheckpoint['state_dict'])
    opt.load_state_dict(mycheckpoint['optimizer'])

In [14]:
testmodel = dense().to(device)

In [16]:
# for x,y in training:
#     x = x.to(device)
#     print(testmodel(x).shape)
#     print(testmodel(x))
#     a = testmodel(x)
# #     pred = [1 if a>0.5 else 0 for a in testmodel(x)]
# #     print(pred)
#     break

In [20]:
# aa = (a>0.5).float()
# aa

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0.],
        [1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0.,
         0.],
        [1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
         0.],
        [0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0.],
        [0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
         0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0.,
         0.],
        [0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0.,
         0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0.,
         0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [22]:
def fit(epoch,train,valid,model,opt,loss_fun,load):
    model.train()
    if load==True:
        load_checkpoint(torch.load('assignment2_rework.pth.tar'))
    
    
    for x in range(epoch):
        
        if (x+1)%2==0:
            check = {'state_dict':model.state_dict(),'optimizer':opt.state_dict()}
            save_checkpoint(check)
        
        accuracy=0
        #aaa=0
        for aaa,(data,target) in enumerate(train):
            #aaa+=1
            data,target = data.to(device),target.to(device)
            pred = model(data)
            loss = loss_fun(pred,target)
            loss.backward()
            opt.step()
            opt.zero_grad()
            prediction = (pred>0.5).float()
            acc=0
            for i in range(len(prediction)):
                if torch.all(prediction[i].eq(target[i])):
                    acc+=1
            accuracy += acc 
            accuracy_rate = acc/batch_size
            if (aaa+1)%10==0:
                print("epoch[{}/{}] batch {} with number of correct prediction {} batch accuracy {:.2%} and loss{:.3f}".format(x+1,epoch,aaa+1,acc,accuracy_rate,loss))
        print("Total Epoch [{}/{}] with accuracy of {:.2%} and loss {:.3f}".format(x+1,epoch,accuracy/25000,loss))
            

In [23]:
loss_fun = nn.BCEWithLogitsLoss()
opt = torch.optim.Adam(testmodel.parameters(),lr=0.0001)

In [24]:
fit(10,training,validation,testmodel,opt,loss_fun,load=True)

:) loading checkpoint
epoch[1/10] batch 10 with number of correct prediction 13 batch accuracy 52.00% and loss0.125
epoch[1/10] batch 20 with number of correct prediction 13 batch accuracy 52.00% and loss0.130
epoch[1/10] batch 30 with number of correct prediction 8 batch accuracy 32.00% and loss0.168
epoch[1/10] batch 40 with number of correct prediction 9 batch accuracy 36.00% and loss0.167
epoch[1/10] batch 50 with number of correct prediction 12 batch accuracy 48.00% and loss0.162
epoch[1/10] batch 60 with number of correct prediction 12 batch accuracy 48.00% and loss0.168
epoch[1/10] batch 70 with number of correct prediction 10 batch accuracy 40.00% and loss0.143
epoch[1/10] batch 80 with number of correct prediction 16 batch accuracy 64.00% and loss0.130
epoch[1/10] batch 90 with number of correct prediction 11 batch accuracy 44.00% and loss0.150
epoch[1/10] batch 100 with number of correct prediction 4 batch accuracy 16.00% and loss0.203
epoch[1/10] batch 110 with number of cor

epoch[2/10] batch 700 with number of correct prediction 8 batch accuracy 32.00% and loss0.157
epoch[2/10] batch 710 with number of correct prediction 10 batch accuracy 40.00% and loss0.144
epoch[2/10] batch 720 with number of correct prediction 11 batch accuracy 44.00% and loss0.155
epoch[2/10] batch 730 with number of correct prediction 7 batch accuracy 28.00% and loss0.181
epoch[2/10] batch 740 with number of correct prediction 10 batch accuracy 40.00% and loss0.194
epoch[2/10] batch 750 with number of correct prediction 8 batch accuracy 32.00% and loss0.190
epoch[2/10] batch 760 with number of correct prediction 12 batch accuracy 48.00% and loss0.148
epoch[2/10] batch 770 with number of correct prediction 11 batch accuracy 44.00% and loss0.153
epoch[2/10] batch 780 with number of correct prediction 6 batch accuracy 24.00% and loss0.197
epoch[2/10] batch 790 with number of correct prediction 12 batch accuracy 48.00% and loss0.134
epoch[2/10] batch 800 with number of correct predictio

KeyboardInterrupt: 

In [18]:
with torch.no_grad():
    testmodel.eval()
    acc=0
    for data,target in validation:
        data,target = data.to(device),target.to(device)
        pred=testmodel(data)
        prediction = (pred>0.5).float()
        for i in range(len(prediction)):
            if torch.all(prediction[i].eq(target[i])):
                acc+=1
    print("Validation accuray is {:.2%}".format(acc/5000))

Validation accuray is 42.00%
